## Computer Vision 이상치 탐지 알고리즘 경진대회

#### 직접 모델 설계 후 학습을 진행했으나 Loss가 많이 떨어지지 않았음

In [1]:
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchsummary import summary

import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import numpy as np
import cv2
from terminaltables import AsciiTable

### 증상 정리

color : 제품에 부자연스러운 색깔이 첨가됨(예 : 알약에 빨간점 찍힘) <br>
scratch : 제품에 흠집이 있음 <br>
crack : 제품이 어딘가 깨짐 <br>
combined : 여러 증상이 같이 있는 제품 <br>

### 느낀점

1. 데이터셋의 product는 괜찮은데 state가 좋지않다. product만의 고유 state가 각자 존재하는듯 하다. 
2. 제품의 상태가 괜찮으면 'good'이라는 state로 설정되는데 이건 전제품 공통이고 이상증세는 제품별로 다른 경우가 있고 같은 경우가 있다. 
   예를 들어 약과 헤이즐넛은 'crack'이라는 공통된 상태를 가지지만 crack이라는 상태가 모든 제품에 존재하는 state인건 아니다. 
3. '제품' 분류는 가능하겠는데 '상태' 분류가 힘들듯. 어떻게 하지? 
4. state의 종류는 49가지. 그리고 생각해봤는데 알약의 crack과 헤이즐넛의 crack은 추출될 feature의 종류가 다를거란 말이지...
5. 그러면 '제품'이라는 방을 먼저 고르고 그 안에서 제품의 상태를 판단하게 하는건 어떨까? 
6. product별로 softmax 시도

## Create Dataloader

In [2]:
class Product_State_Classification_Dataset(Dataset) :
    def __init__(self, dataset_root, image_name_list, label_list, label_index_list): 
        
        self.dataset_root = dataset_root
        self.image_name_list = image_name_list
        self.label_list = label_list
        self.label_index_list = label_index_list
        
    def __len__(self): 
        return len(self.image_name_list) 
    
    def __getitem__(self, index): 
        
        # image processing
        to_tensor = transforms.ToTensor()
        img = np.array(Image.open(self.dataset_root + self.image_name_list[index]).convert('RGB'))
        img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
        # Normalize 연산할 때 기존 방식인 '255로 나누기'를 수행하는 과정에서 NaN이 발생. opencv서서 제공하는 함수로 normalize 처리하니 NaN이 발생하지 않음 
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        img = to_tensor(img)        
        
        # label data
        label = np.where(self.label_index_list == self.label_list[index])[0][0]
        
        return img, torch.tensor(label)

In [3]:
def make_Dataset(dataset_root) :
    csv = pd.read_csv(dataset_root + 'train_df.csv')
    
    image_name_list = csv['file_name'].tolist()
    label_list = csv['label'].tolist()
    
    label_index_list = np.unique(csv['label'].tolist())
    
    dataset = Product_State_Classification_Dataset(dataset_root + 'train/', image_name_list, label_list, label_index_list)
    
    return dataset, label_index_list

In [4]:
def make_DataLoader(dataset_root, BATCH_SIZE = 64) :
    dataset, label_index_list = make_Dataset(dataset_root)
    
    DataLoader = torch.utils.data.DataLoader(dataset = dataset, # 사용할 데이터셋
                                          batch_size = BATCH_SIZE, # 미니배치 크기
                                          shuffle=True, # 에포크마다 데이터셋 셔플할건가? 
                                          num_workers=12,
                                          pin_memory=False,
                                          drop_last=True)
    
    return DataLoader, label_index_list

## Create Model

In [5]:
class ResidualBlock(nn.Module) :
    def __init__(self, channel) :
        super(ResidualBlock, self).__init__()
        
        self.model = nn.Sequential(
            nn.Conv2d(channel, channel, kernel_size=3, stride = 1, padding = 1, bias=False),
            # nn.BatchNorm2d(channel),
            nn.LeakyReLU(inplace=False),
            nn.Conv2d(channel, channel, kernel_size=3, stride = 1, padding = 1, bias=False),
            # nn.BatchNorm2d(channel)
        )
        self.LeakyReLU = nn.LeakyReLU(inplace=False)
        
    
    def forward(self, x) :
        output = self.model(x)
        
        output += x
        
        output = self.LeakyReLU(output)
        
        return output
    
class ResNet(nn.Module) :
    def __init__(self) :
        super(ResNet, self).__init__()        
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            ResidualBlock(64), # 2_1
            ResidualBlock(64), # 2_2
            nn.Conv2d(64, 128, kernel_size=1, stride = 2, padding = 0, bias=False),
            ResidualBlock(128), # 3_1
            ResidualBlock(128), # 3_2
            ResidualBlock(128), # 3_3
            nn.Conv2d(128, 256, kernel_size=1, stride = 2, padding = 0, bias=False),
            ResidualBlock(256), # 4_1
            ResidualBlock(256), # 4_2
            ResidualBlock(256), # 4_3
            nn.Conv2d(256, 512, kernel_size=1, stride = 2, padding = 0, bias=False),
            ResidualBlock(512), # 5_1
            ResidualBlock(512), # 5_2
            ResidualBlock(512), # 5_3
            nn.AvgPool2d(3, stride=2),
            nn.Flatten()
        )
        
        self.linear = nn.Sequential(
            nn.Linear(4608, 88, bias = False),
            nn.Softmax(dim=1)
        )

    def forward(self, x) :
        cnn_output = self.cnn(x)
        output   = self.linear(cnn_output)
        
        return output

In [6]:
# cnn = nn.Sequential(
#             nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
#             nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
#             ResidualBlock(64), # 2_1
#             ResidualBlock(64), # 2_2
#             ResidualBlock(64), # 2_3
#             nn.Conv2d(64, 128, kernel_size=1, stride = 2, padding = 0, bias=False),
#             ResidualBlock(128), # 3_1
#             ResidualBlock(128), # 3_2
#             ResidualBlock(128), # 3_3
#             ResidualBlock(128), # 3_4
#             nn.Conv2d(128, 256, kernel_size=1, stride = 2, padding = 0, bias=False),
#             ResidualBlock(256), # 4_1
#             ResidualBlock(256), # 4_2
#             ResidualBlock(256), # 4_3
#             ResidualBlock(256), # 4_4
#             ResidualBlock(256), # 4_5
#             ResidualBlock(256), # 4_6
#             nn.Conv2d(256, 512, kernel_size=1, stride = 2, padding = 0, bias=False),
#             ResidualBlock(512), # 5_1
#             ResidualBlock(512), # 5_2
#             ResidualBlock(512), # 5_3
#             nn.AvgPool2d(3, stride=2),
#             nn.Flatten()
#         ).to('cuda')
# summary(cnn, input_size = (3, 224, 224))

## Train, Test model

In [7]:
def get_accuracy(pred, target) :
    
    pred = torch.argmax(pred, dim=1)
    
    compare_result = torch.eq(pred, target).type(torch.LongTensor)
    
    return torch.sum(compare_result).item(), pred.size()[0]

def train_model(model, loss_function, train_DataLoader, EPOCH=300) : 
    print("\n---- Training Model ----")
    
    # optimizer 설정
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 1e-4)
    
    model_weights_root_folder = '/home/minkyukim/models/Dacon_model/'
    max_acc_epoch = None
    max_acc = None
        
    model.train()
    # 코드 출처 : https://github.com/eriklindernoren/PyTorch-YOLOv3
    for epoch in range(EPOCH):

        epoch_loss = 0.0
        
        epoch_acc = 0.0
        epoch_answer_count = 0
        epoch_data_num = 0
        
        checkpoint_path = model_weights_root_folder + 'model_weights/dacon_model_epoch_' + str(epoch) + '.pth'
        
        pbar = tqdm(train_DataLoader, desc=f"Training Epoch {epoch}")
        
        batch_i = 1
        train_data_num = 0.0
        
        for _, (imgs, targets) in enumerate(pbar):
            batches_done = len(train_DataLoader) * epoch + batch_i 
            imgs = imgs.to('cuda', non_blocking=True)
            targets = targets.to('cuda')
            with torch.autograd.set_detect_anomaly(True) : 
                preds = model(imgs)
                train_loss = loss_function(preds, targets) # 나중에 loss를 저장 후 print_loss()에서 출력
                answer_count_per_batch, batch_num = get_accuracy(preds.clone().detach().requires_grad_(False), targets)
            
                train_loss.backward()
                    
#             if batches_done >= (len(train_DataLoader) * 500) :
#                 for g in optimizer.param_groups:
#                     g['lr'] /=10
                
            optimizer.step()
            optimizer.zero_grad()
            
            batch_i += 1 # 학습에 관여한 batch개수를 알기 위해 사용하는 변수
            epoch_loss += train_loss.item() # epoch별 평균 loss를 알기 위해 사용하는 변수
            epoch_answer_count += answer_count_per_batch
            epoch_data_num += batch_num
        
        torch.save(model.state_dict(), checkpoint_path) 
        
        epoch_loss /= len(train_DataLoader)
        
        epoch_acc = float(epoch_answer_count) / epoch_data_num
        
        print(AsciiTable(
                [
                    ["Type", "Value"],
                    ["train loss", format(round(epoch_loss, 6), 'f')],
                    ["train acc ", format(round(epoch_acc, 6), 'f')]
                ]).table.replace('+', '|'))
        
        if max_acc == None or max_acc < epoch_acc :
            max_acc_epoch = epoch
            max_acc = epoch_acc
    
    return max_acc_epoch         

In [8]:
def load_model(epoch) :
    weight_root = '/home/minkyukim/models/Dacon_model/model_weights/'
    
    model = ResNet().to('cuda')
    
    model.load_state_dict(torch.load(weight_root + 'dacon_model_epoch_' + str(epoch) + '.pth', map_location='cuda'))
    
    return model

In [9]:
def get_output(model, dataset_root, label_index_list) :
    
    model.eval()
    
    csv = pd.read_csv(dataset_root + 'test_df.csv')
    
    submission = open('/home/minkyukim/models/Dacon_model/submission.csv', 'w')
    submission.write('index,label\n')
    
    image_name_list = csv['file_name'].tolist()
    to_tensor = transforms.ToTensor()
    
    pbar = tqdm(image_name_list, desc = "testing ", mininterval=0.01)
    
    pred_list = []
    
    count = 0
    for image_name in pbar : 
        img = np.array(Image.open(dataset_root + 'test/' + image_name).convert('RGB'))
        img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
        # Normalize 연산할 때 기존 방식인 '255로 나누기'를 수행하는 과정에서 NaN이 발생. opencv서서 제공하는 함수로 normalize 처리하니 NaN이 발생하지 않음 
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        img = torch.unsqueeze(to_tensor(img), 0).to('cuda')       
        with torch.no_grad():
            pred = label_index_list[torch.argmax(torch.squeeze(model(img))).item()]
        print_str = str(count) + "," + pred + '\n'
        submission.write(print_str)
        count+=1
        
    submission.close()
        
    return submission
        

## 실행 코드

In [10]:
dataset_root = '/home/minkyukim/Dataset/Dacon_dataset/'
DataLoader, label_index_list = make_DataLoader(dataset_root)

model = ResNet().to('cuda')
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
         MaxPool2d-2           [-1, 64, 56, 56]               0
            Conv2d-3           [-1, 64, 56, 56]          36,864
         LeakyReLU-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
         LeakyReLU-6           [-1, 64, 56, 56]               0
     ResidualBlock-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
         LeakyReLU-9           [-1, 64, 56, 56]               0
           Conv2d-10           [-1, 64, 56, 56]          36,864
        LeakyReLU-11           [-1, 64, 56, 56]               0
    ResidualBlock-12           [-1, 64, 56, 56]               0
           Conv2d-13          [-1, 128, 28, 28]           8,192
           Conv2d-14          [-1, 128,

/home/minkyukim/anaconda3/envs/PyTorch_p39/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [11]:
loss_function = nn.CrossEntropyLoss()
max_acc_epoch = train_model(model, loss_function, DataLoader)


---- Training Model ----


Training Epoch 0: 100%|█████████████████████████| 66/66 [00:18<00:00,  3.49it/s]


|------------|----------|
| Type       | Value    |
|------------|----------|
| train loss | 4.425089 |
| train acc  | 0.072680 |
|------------|----------|


Training Epoch 1: 100%|█████████████████████████| 66/66 [00:19<00:00,  3.44it/s]


|------------|----------|
| Type       | Value    |
|------------|----------|
| train loss | 4.421390 |
| train acc  | 0.075284 |
|------------|----------|


...

|------------|----------|
| Type       | Value    |
|------------|----------|
| train loss | 4.404819 |
| train acc  | 0.091856 |
|------------|----------|


Training Epoch 298: 100%|███████████████████████| 66/66 [00:18<00:00,  3.56it/s]


|------------|----------|
| Type       | Value    |
|------------|----------|
| train loss | 4.405293 |
| train acc  | 0.091383 |
|------------|----------|


Training Epoch 299: 100%|███████████████████████| 66/66 [00:18<00:00,  3.51it/s]


|------------|----------|
| Type       | Value    |
|------------|----------|
| train loss | 4.405766 |
| train acc  | 0.090909 |
|------------|----------|


In [12]:
model = load_model(max_acc_epoch)
submission = get_output(model, dataset_root, label_index_list)

testing : 100%|█████████████████████████████| 2154/2154 [01:12<00:00, 29.76it/s]
